In [1]:
import numpy as np

import sensors
import em_stoch

In [2]:
DIST_RATIO = 0.5

In [3]:
Num_sensors1 = 5
Num_emitters1 = 1
sample_size1 = 100
theta1_rad = [0.7] # Угловые координаты источников (DoA) в радианах
theta1_deg = np.rad2deg(theta1_rad) # Угловые координаты источников (DoA) в градусах
P_1 = 1 * np.eye(Num_emitters1, dtype=np.float64) # Ковариация сигналов
Q_1 = 1.1 * np.eye(Num_sensors1, dtype=np.float64) # Ковариация шумов
A1 = (np.exp(-2j * np.pi * DIST_RATIO * np.arange(Num_sensors1).reshape(-1,1) * 
             np.sin(theta1_rad))) # Матрица векторов направленности
# Генерация сигналов, шумов и наблюдений
S1 = sensors.gss(Num_emitters1, sample_size1, P_1)
N1 = sensors.gss(Num_sensors1, sample_size1, Q_1)
X1 = (A1 @ S1.T + N1.T).T

Num_sensors2 = 5
Num_emitters2 = 2
sample_size2 = 100
theta2_rad = np.array([-0.5, 0.7], dtype=np.float64) # Угловые координаты источников (DoA) в радианах
theta2_deg = np.rad2deg(theta2_rad) # Угловые координаты источников (DoA) в градусах
P_2 = 1 * np.eye(Num_emitters2, dtype=np.float64) # Ковариация сигналов
Q_2 = 1.21 * np.eye(Num_sensors2, dtype=np.float64) # Ковариация шумов
A2 = (np.exp(-2j * np.pi * DIST_RATIO * np.arange(Num_sensors2).reshape(-1,1) * 
             np.sin(theta2_rad))) # Матрица векторов направленности
# Генерация сигналов, шумов и наблюдений
S2 = sensors.gss(Num_emitters2, sample_size2, P_2)
N2 = sensors.gss(Num_sensors2, sample_size2, Q_2)
X2 = (A2 @ S2.T + N2.T).T

In [4]:
X1_with_mv = sensors.MCAR(X1, [2,4], [0.4, 0.4])
X2_with_mv = sensors.MCAR(X2, [2,4], [0.4, 0.4])

In [5]:
X1_with_mv

array([[ 1.23286261e+00+0.9418559j , -1.85659807e-02-2.84696988j,
                    nan+0.j        ,  2.32727443e-01-0.74680402j,
                    nan+0.j        ],
       [-1.84555688e+00-0.03615284j,  5.79029362e-01+0.98613426j,
         2.52068362e-01-0.41735757j, -8.97514167e-01+0.04994758j,
                    nan+0.j        ],
       [-1.09664728e+00+1.1646421j , -1.72871621e-01-0.7934921j ,
         1.76743201e+00-1.60581369j, -1.22187416e+00+0.6652129j ,
                    nan+0.j        ],
       [-3.19705011e+00-1.03130086j,  6.19777131e-01+1.91974302j,
         2.07216841e+00-3.48689659j, -2.50062242e+00-0.98218031j,
        -3.00018978e-01+1.93779552j],
       [-1.05663395e+00-0.81800175j, -7.86290658e-01+1.43634167j,
                    nan+0.j        , -3.21607249e-01-0.98971863j,
         3.63068447e-01+1.76799945j],
       [-4.64436551e-01+1.06533348j,  4.39830960e-01+1.36970282j,
                    nan+0.j        , -1.05030240e+00+0.27738488j,
                  

In [6]:
print(np.trace(sensors.initial_Cov(X1_with_mv)))
print(np.trace(sensors.complex_cov(X1)))

(10.280950744938718+0j)
(11.105494677649398+0j)


In [7]:
EM_theta1_rad, EM_P1, lhd_1 = em_stoch.multi_start_EM(X1, Num_emitters1, Q_1, 10, 
                                                         max_iter=30, 
                                                         rtol=1e-6)

0-th start
theta=[0.15335215],P=[0.12062233-1.20247748e-17j]
res=[[0.12062227-1.02900513e-17j]]
new_angles=[0.15335215]
1-th start
theta=[0.13636065],P=[0.14490204-3.87895961e-17j]
res=[[0.14490196-4.22590431e-17j]]
new_angles=[0.13636065]
2-th start
theta=[1.40627821],P=[0.09965251+8.53371115e-18j]
res=[[0.09965247+1.02684346e-17j]]
new_angles=[1.40627821]
3-th start
theta=[-0.15355298],P=[0.00044721+0.j]
res=[[0.00044721+5.29395592e-23j]]
new_angles=[-0.15355298]
4-th start
theta=[0.5300775],P=[1.72866676+1.11714037e-16j]
res=[[1.72866208-2.76864022e-16j]]
new_angles=[0.5300775]
likelihood is -874.9333570991064 on iteration 0
res=[[1.72865741+0.j]]
new_angles=[0.57340969]
likelihood is -813.6916432175549 on iteration 1
res=[[1.72865466-7.77156117e-16j]]
new_angles=[0.60578727]
likelihood is -778.0680302490574 on iteration 2
res=[[1.72865301-2.22044605e-16j]]
new_angles=[0.62962035]
likelihood is -758.6037033193436 on iteration 3
res=[[1.72865197-1.2490009e-16j]]
new_angles=[0.6470192

In [8]:
print(EM_theta1_rad, EM_P1)

[0.69244072] [[1.72864218]]


In [9]:
EM_theta1_deg = np.rad2deg(EM_theta1_rad)
print(f"Погрешность в смысле разности углов в градусах: \
      {np.abs(EM_theta1_deg-theta1_deg)},")
print(f"Погрешность в смысле разности синусов углов: \
      {np.abs(np.sin(EM_theta1_rad)-np.sin(theta1_rad))}.")

Погрешность в смысле разности углов в градусах:       [0.43311459],
Погрешность в смысле разности синусов углов:       [0.0058].


In [10]:
EM_theta2_rad, EM_P2, lhd_2 = em_stoch.multi_start_EM(X2_with_mv, Num_emitters2, Q_2, 10, 
                                                         max_iter=30, 
                                                         rtol=1e-3)

0-th start
theta=[-1.41744418  0.15335215],P=[0.00044721+0.00000000e+00j 0.23768273+3.68364405e-18j]
res=[[0.00044696+5.30230700e-23j 0.        +0.00000000e+00j]
 [0.        +0.00000000e+00j 0.18789777+8.16445417e-19j]]
new_angles=[-1.41744419  0.15944998]
likelihood is -1178.1134706506416 on iteration 0
res=[[0.00044684+1.48254959e-23j 0.        +0.00000000e+00j]
 [0.        +0.00000000e+00j 0.1763593 -2.09797102e-19j]]
new_angles=[-1.41744421  0.14646935]
likelihood is -1176.426452625598 on iteration 1
res=[[0.00044672+7.13927777e-24j 0.        +0.00000000e+00j]
 [0.        +0.00000000e+00j 0.16605908+3.84990856e-19j]]
new_angles=[-1.41744422  0.13599647]
likelihood is -1175.37734933355 on iteration 2
res=[[0.00044661+1.11812769e-23j 0.        +0.00000000e+00j]
 [0.        +0.00000000e+00j 0.15727044-2.02761348e-19j]]
new_angles=[-1.41744424  0.12751633]
likelihood is -1174.767464247413 on iteration 3
res=[[0.00044652+1.33615229e-23j 0.        +0.00000000e+00j]
 [0.        +0.0000000

c:\Users\asus\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\optimize\_slsqp_py.py:435: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)


new_angles=[-0.57997339  0.66634514]
likelihood is -950.6544052508126 on iteration 11
res=[[0.05430348-5.50203331e-20j 0.        +0.00000000e+00j]
 [0.        +0.00000000e+00j 1.04623675-6.02077547e-17j]]
new_angles=[-0.57239136  0.66810493]
likelihood is -949.878674668555 on iteration 12
res=[[0.05407419-1.80895277e-20j 0.        +0.00000000e+00j]
 [0.        +0.00000000e+00j 1.02197027+8.91201634e-17j]]
new_angles=[-0.56623302  0.66967759]
likelihood is -949.3482121273354 on iteration 13
res=[[0.05386751-3.31074233e-20j 0.        +0.00000000e+00j]
 [0.        +0.00000000e+00j 0.99966857+6.88377654e-17j]]
new_angles=[-0.5612073   0.67108589]
likelihood is -948.9965838384117 on iteration 14
res=[[0.05367884+1.39195128e-20j 0.        +0.00000000e+00j]
 [0.        +0.00000000e+00j 0.97907612+2.17303353e-17j]]
new_angles=[-0.5570902   0.67235004]
likelihood is -948.7770771235599 on iteration 15
res=[[0.05350484-2.08889436e-19j 0.        +0.00000000e+00j]
 [0.        +0.00000000e+00j 0.959

In [11]:
print(EM_theta2_rad, EM_P2)

[-0.53968017  0.68167924] [[0.05193081 0.        ]
 [0.         0.79448743]]


In [12]:
EM_theta2_deg = np.rad2deg(EM_theta2_rad)
print(f"Погрешность в смысле разности углов в градусах: \
      {np.abs(np.sort(EM_theta2_deg)-np.sort(theta2_deg))},")
print(f"Погрешность в смысле разности синусов углов: \
      {np.abs(np.sin(EM_theta2_rad)-np.sin(theta2_rad))}.")

Погрешность в смысле разности углов в градусах:       [2.27350605 1.04970238],
Погрешность в смысле разности синусов углов:       [0.0344361  0.01411982].


In [13]:
em_stoch.incomplete_lkhd(X1_with_mv, theta1_rad, 1.1*P_1, Q_1)

np.float64(-615.0813000159774)

In [14]:
em_stoch.incomplete_lkhd(X2_with_mv, theta2_rad, P_2, Q_2)

np.float64(-784.7959062751833)